# Working with YAML files
- YAML (“YAML Ain’t Markup Language”) **focuses on human readability**. Indentation replaces braces and brackets, **comments are allowed**, and **quoting is usually optional**. This makes it easier to document and manage complex configuration files. 
- DevOps tooling (Kubernetes, Ansible, GitHub Actions, many app configs) standardizes on YAML for its clarity and brevity.
- JSON is excellent for machine-to-machine communication, but its strict syntax (**no comments**, heavy quoting) can feel verbose to humans maintaining config files.
- Python’s standard library lacks YAML support; **PyYAML** is the community-standard package to fill that gap.

## YAML Syntax and Features
- Structure comes from **spaces for indentation**: tabs are discouraged.  
- **Mappings** use `key: value`; **sequences** use a leading hyphen (`-`) plus a space.
- Scalars include strings, numbers, booleans (`true / false`, `yes / no`), and `null`.  
- Comments begin with `#`.
- Multi-line scalars can be literal (`|`) or folded (`>`).  
- **Anchors (&) and aliases (*)** avoid repetition by re-using defined blocks.  
- YAML is a superset of JSON: most valid JSON documents are also valid YAML.

In [9]:
import yaml, json

snippet = """
service: &svc
  name: user-api
  port: 8080
  enabled: true
  tags:
    - api
    - user
    - internal
staging:
  <<: *svc
  replicas: 2
production:
  <<: *svc
  replicas: 4
"""

# Deserialize YAML string to Python object
parsed = yaml.safe_load(snippet)
print(parsed)

# To visual JSON
print(json.dumps(parsed, indent=2))

multiline_demo = """
literal: |
  line 1
  line 2
  line 3
folded: >
  This is a long string that
  could go out of screen, so
  we will break this up into
  multiple lines to improve
  readability.
"""

# Deserialize YAML string to Python object
parsed1 = yaml.safe_load(multiline_demo)

# To visual JSON
print(json.dumps(parsed1, indent=2))

{'service': {'name': 'user-api', 'port': 8080, 'enabled': True, 'tags': ['api', 'user', 'internal']}, 'staging': {'name': 'user-api', 'port': 8080, 'enabled': True, 'tags': ['api', 'user', 'internal'], 'replicas': 2}, 'production': {'name': 'user-api', 'port': 8080, 'enabled': True, 'tags': ['api', 'user', 'internal'], 'replicas': 4}}
{
  "service": {
    "name": "user-api",
    "port": 8080,
    "enabled": true,
    "tags": [
      "api",
      "user",
      "internal"
    ]
  },
  "staging": {
    "name": "user-api",
    "port": 8080,
    "enabled": true,
    "tags": [
      "api",
      "user",
      "internal"
    ],
    "replicas": 2
  },
  "production": {
    "name": "user-api",
    "port": 8080,
    "enabled": true,
    "tags": [
      "api",
      "user",
      "internal"
    ],
    "replicas": 4
  }
}
{
  "literal": "line 1\nline 2\nline 3\n",
  "folded": "This is a long string that could go out of screen, so we will break this up into multiple lines to improve readability.\n"

## Deserializing YAML with `yaml.safe_load`
- Prefer **`yaml.safe_load`** (or passing `Loader=yaml.SafeLoader`) to prevent arbitrary-code execution; avoid `yaml.load` on untrusted data.
- Accepts a string or an open text file handle and returns native Python structures.  
- Wrap calls in `try / except yaml.YAMLError` to catch malformed input.

In [10]:
import yaml, json
from pathlib import Path

compose = Path("files/compose.yaml")

try:
    with compose.open("r", encoding="utf-8") as file:
        config = yaml.safe_load(file)
        print(f"Compose version: {config["version"]}")
    
        for svc, options in config["services"].items():
            print(f"{svc.capitalize()} image\t: {options["image"]}")
except yaml.YAMLError as e:
    print("YAML error:")
    print(e)

# For visualization
print(json.dumps(config, indent=2))


Compose version: 3.8
Web image	: myapp:latest
Redis image	: redis:alpine
{
  "version": "3.8",
  "services": {
    "web": {
      "image": "myapp:latest",
      "ports": [
        "8000:80"
      ]
    },
    "redis": {
      "image": "redis:alpine"
    }
  }
}


## Serializing Python Objects with `yaml.dump`
- Use `yaml.dump(obj, indent=2, default_flow_style=False, sort_keys=False)` for readable block-style output.  
- Set `stream` to an open file handle to write directly; leave it `None` to return a string.

In [12]:
import yaml
from pathlib import Path

python_cfg = {
    "service": {"name": "listener-service", "port": 6789, "workers": 4, "enabled": False},
    "queues": ["high", "default", "low"],
    "retry_policy": None,
}

output_path = Path("files/listener_config.yaml")

with output_path.open("w", encoding="utf-8") as file:
    yaml.dump(python_cfg, file, sort_keys=False, default_flow_style=False)